In [51]:
from kokoro import KPipeline
import soundfile as sf
from pydub import AudioSegment
import json;
import time;
import mysql.connector;
import requests
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import os


In [52]:
mysql_host = '192.168.0.202'
mysql_port = 4000
file_app_url = "http://192.168.0.202:3007/file"
hour = 24
get_sql = f"select news_id, news_company from news where date >= NOW() - INTERVAL {hour} HOUR;"

In [53]:
db_conn = mysql.connector.connect(
    host=mysql_host,
    port=mysql_port,
    user="root",
    password="password",
    database="news"
)
cursor = db_conn.cursor(dictionary=True)
cursor.execute(get_sql)
rows = cursor.fetchall()

In [ ]:
news_id = list()
for r in rows:
  news_id.append(r['news_id'])

In [55]:
news_id

[42553, 42953, 43113, 43233, 43253, 43513, 44073]

In [56]:
def download_wav(id):
    url = f"{file_app_url}?fileName={id}.wav"
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            filename = f"{id}.wav"
            with open(filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            print(f"Downloaded {filename}")
        else:
            print(f"Failed to download {id}.wav, status code: {response.status_code}")
    except Exception as e:
        print(f"Error downloading {id}.wav: {e}")

In [57]:
with ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(download_wav, news_id)

Downloaded 42553.wav
Downloaded 42953.wav
Downloaded 43233.wav
Downloaded 43113.wav
Downloaded 43253.wav
Downloaded 43513.wav
Downloaded 44073.wav


In [58]:
output = AudioSegment.empty()
for i in news_id:  # include 0.wav to n.wav
    output += AudioSegment.from_wav("next.wav")
    file_name = f"{i}.wav"
    audio = AudioSegment.from_wav(file_name)
    output += audio  # append

output.export("output.wav", format="wav")

<_io.BufferedRandom name='output.wav'>

In [ ]:
duration = int(len(output) / 1000.0 / 60)

In [60]:
def get_greeting(hour):
    if 5 <= hour < 12:
        return "Good morning"
    elif 12 <= hour < 18:
        return "Good afternoon"
    elif 18 <= hour < 22:
        return "Good evening"
    else:
        return "Good night"

In [61]:
greeting = get_greeting(datetime.now().hour)
date_str = datetime.now().strftime("%B %d, %Y")
total_news = len(rows)

In [62]:
companies = {row['news_company'] for row in rows if row['news_company']}
company_str = ", ".join(companies) if companies else "various sources"

In [63]:
message = f"{greeting}! Today is {date_str}. There are {total_news} news articles from {company_str} in the past {hour} hours. It will take about {duration} minutes to go through them."

In [64]:
message

'Good night! Today is April 25, 2025. There are 7 news articles from nytime in the past 24 hours. It will take about 10 minutes to go through them.'

In [65]:
pipeline = KPipeline(lang_code='a')
generator = pipeline(message, voice='af_heart')
n = 0
for i, (gs, ps, audio) in enumerate(generator):
    n = i
    print(i, gs, ps)
    sf.write(f'{i}.wav', audio, 24000)

c:\Users\16507\anaconda3\envs\TTS\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
c:\Users\16507\anaconda3\envs\TTS\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


0 Good night! Today is April 25, 2025. There are 7 news articles from nytime in the past 24 hours. It will take about 10 minutes to go through them. ɡˈʊd nˈIt! tədˈA ɪz ˈApɹəl twˈɛnti fˈIv, twˈɛnti twˈɛnti fˈIv. ðˌɛɹ ɑɹ sˈɛvən nˈuz ˈɑɹTəkᵊlz fɹʌm nˈItIm ɪn ðə pˈæst twˈɛnti fˈɔɹ ˈWəɹz. ˌɪt wɪl tˈAk əbˈWt tˈɛn mˈɪnəts tə ɡˌO θɹu ðˌɛm.


In [66]:
output = AudioSegment.empty()
for i in range(n + 1):  # include 0.wav to n.wav
    file_name = f"{i}.wav"
    audio = AudioSegment.from_wav(file_name)
    output += audio  # append

output += AudioSegment.from_wav("output.wav")
output += AudioSegment.from_wav("end.wav")

output.export("daily.wav", format="wav")

<_io.BufferedRandom name='daily.wav'>

In [67]:
files = {
  "file": open("daily.wav", "rb")
}

data = {
  "fileName": "daily",
  "fileType": "wav"
}

response = requests.post(file_app_url, files=files, data=data)

In [68]:
for id in news_id:
    filename = f"{id}.wav"
    if os.path.exists(filename):
        try:
            os.remove(filename)
            print(f"Deleted {filename}")
        except Exception as e:
            print(f"Error deleting {filename}: {e}")

Deleted 42553.wav
Deleted 42953.wav
Deleted 43113.wav
Deleted 43233.wav
Deleted 43253.wav
Deleted 43513.wav
Deleted 44073.wav
